<a href="https://colab.research.google.com/github/zaidishz/GNN_XAI_Captum/blob/main/GNN_Explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explaining GNN Model Predictions using Captum

Mutagenicity dataset from [TUDatasets](https://chrsmrrs.github.io/datasets/). This dataset consists of 4337 molecule graphs where the task is to predict the molecule mutagenicity.

In [1]:
# Install required packages.
import os
from torch import __version__, tensor
os.environ['TORCH'] = __version__
print(__version__)

%matplotlib inline
import matplotlib.pyplot as plt
import scipy.io
import numpy as  np
import itertools
from sklearn.preprocessing import StandardScaler
from scipy.fft import fft, fftfreq
import pandas as pd

2.0.0+cpu


## Loading the dataset

load the dataset (10% of the data as the test split)

In [2]:
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
import os

proxy = 'http://webproxy:8080'

os.environ['http_proxy'] = proxy
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [3]:
achse_mat = scipy.io.loadmat('C:\\Users\\vedurm01\\PycharmProjects\\ZeMA_ML\\ZeMA_Matlab\\Achse11_Szenario.mat')

trainData = achse_mat['trainData']
train_data_reshaped = []
for counter, val in enumerate(trainData):
    train_data_reshaped += [val[0]]
train_data_reshaped = np.array(train_data_reshaped)
train_data_reshaped = np.swapaxes(train_data_reshaped, 0, 1)

train_target_reshaped = achse_mat['trainTarget'].reshape(-1)

C:\Users\vedurm01\PycharmProjects\GNN_XAI_Captum\venv\lib\site-packages\scipy\io\matlab\_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [4]:
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))

## Convert time-series data to torch geometric graph data. Fully connected graph with 55 edges. Each edge has the correlation between the sensors as an attribute

In [5]:
dataset = []
edge_index_perm = tensor(np.array(list(itertools.combinations(range(1,12), 2))).T)
triu_ind = np.triu_indices(11, k = 1)
N = 2000
Tsig = 1 / 2000.0
seconds = 5
fps = 50
for k, sample in enumerate(train_data_reshaped):
#     offset = []
#     for k in range(10):
#         for l in range(k+1,11):
#             d1 = pd.Series(sample[0][k])
#             d2 = pd.Series(sample[0][l])
#             rs = [crosscorr(d1, d2, lag) for lag in range(-int(seconds*fps),int(seconds*fps+1))]
#             offset.append(np.floor(len(rs)/2)-np.argmax(rs))
    corr_mat = np.corrcoef(sample)
#     edge_attr = np.append(np.reshape(corr_mat[triu_ind], (55,-1)), np.array(offset).reshape(55,-1), axis=1)
    edge_attr = np.reshape(corr_mat[triu_ind], (55,-1))
    scaler = StandardScaler()
    array = scaler.fit_transform(sample.T).T
    array_fft = np.abs(fft(array)[:N//2])
    dataset += [Data(x=tensor(array_fft), edge_index=edge_index_perm, edge_attr= tensor(edge_attr), y=tensor(train_target_reshaped[k]))]
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

In [6]:
test_loader = DataLoader(test_dataset, batch_size=128)
train_loader = DataLoader(train_dataset, batch_size=128)

## Visualizing the data


In [7]:
import networkx as nx
import numpy as np

from torch_geometric.utils import to_networkx


def draw_molecule(g, edge_mask=None, draw_edge_labels=False):
    g = g.copy().to_undirected()
    node_labels = {}
    for u, data in g.nodes(data=True):
        node_labels[u] = data['name']
    pos = nx.planar_layout(g)
    pos = nx.spring_layout(g, pos=pos)
    if edge_mask is None:
        edge_color = 'black'
        widths = None
    else:
        edge_color = [edge_mask[(u, v)] for u, v in g.edges()]
        widths = [x * 10 for x in edge_color]
    nx.draw(g, pos=pos, labels=node_labels, width=widths,
            edge_color=edge_color, edge_cmap=plt.cm.Blues,
            node_color='azure')
    
    if draw_edge_labels and edge_mask is not None:
        edge_labels = {k: ('%.2f' % v) for k, v in edge_mask.items()}    
        nx.draw_networkx_edge_labels(g, pos, edge_labels=edge_labels,
                                    font_color='red')
    plt.show()


def to_molecule(data):
    ATOM_MAP = ['C', 'O', 'Cl', 'H', 'N', 'F',
                'Br', 'S', 'P', 'I', 'Na', 'K', 'Li', 'Ca']
    g = to_networkx(data, node_attrs=['x'])
    for u, data in g.nodes(data=True):
        data['name'] = ATOM_MAP[data['x'].index(1.0)]
        del data['x']
    return g


In [18]:
import random

data = random.choice([t for t in train_dataset])
print(data)
g = to_networkx(data, node_attrs=['x'])
for u, data in g.nodes(data=True):
    print(data)

Data(x=[11, 2000], edge_index=[2, 55], edge_attr=[55, 1], y=38)


IndexError: list index out of range

## Sample visualization

We sample a single molecule from train_dataset and visualize it

In [8]:
import random

data = random.choice([t for t in train_dataset])
mol = to_molecule(data)
plt.figure(figsize=(10, 5))
draw_molecule(mol)

IndexError: list index out of range

In [ ]:
T=1/2000.0
N=2000
y = fft(train_data_reshaped[0][0])
xf = fftfreq(N, T)[:N//2]

In [ ]:
plt.plot(xf, 2.0/N * np.real(y[0:N//2]))
plt.plot(xf, 2.0/N * np.imag(y[0:N//2]))

In [ ]:
plt.plot(xf, 2.0/N * np.abs(y[0:N//2]))

In [ ]:
plt.plot(train_data_reshaped[500][7])

In [ ]:
plt.scatter(train_data_reshaped[0][5], train_data_reshaped[0][6])

In [ ]:
np.abs(y[0:N//2]).max() / np.abs(y[0:N//2]).min()

In [ ]:
alldat=[]
for k in range(11):
    T=1/2000.0
    N=2000
    y = fft(train_data_reshaped[0][k])
#     plt.scatter((k+1)*np.ones(1000), 2.0/N * np.abs(y[0:N//2]))
    alldat.append(np.abs(y))

In [ ]:
plt.boxplot(alldat)

In [ ]:
df_temp=pd.DataFrame(train_data_reshaped[0].T, columns=['Sensor '+str(k+1) for k in range(11)])
d1 = df_temp['Sensor 5']
d2 = df_temp['Sensor 6']

In [ ]:
triu_ind = np.triu_indices(11, k = 1)
corr_mat = np.corrcoef(train_data_reshaped[0])
corr_arr=np.reshape(corr_mat[triu_ind], (55,-1))

In [ ]:
np.append(corr_arr, np.linspace(1,55,55).reshape(55,-1), axis=1)

In [ ]:
np.linspace(1,55,55)